# 1. Task的优势

ThreadPool相比Thread来说具备了很多优势，但是ThreadPool却又存在一些使用上的不方便。比如：
- ThreadPool不支持线程的取消、完成、失败通知等交互性操作；
- ThreadPool不支持线程执行的先后次序；


以往，如果开发者要实现上述功能，需要完成很多额外的工作，现在，FCL中提供了一个功能更强大的概念：Task。Task在线程池的基础上进行了优化，并提供了更多的API。在FCL4.0中，如果我们要编写多线程程序，Task显然已经优于传统的方式。  
以下是两个简单的任务示例：

In [1]:
using System.Threading;
using System.Threading.Tasks;

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
Task t = new Task(() =>
{
    Console.WriteLine("任务开始工作... ...");
    //模拟工作过程
    Thread.Sleep(5000);
});
t.Start();
t.ContinueWith((task) =>
{
    Console.WriteLine($"任务完成，完成时候的状态为： IsCanceled={task.IsCanceled}  IsCompleted={task.IsCompleted}  IsFaulted={task.IsFaulted}");
});

// Task.WaitAll(t);

//模拟另一个任务
for (int i = 7; i > 0; i--)
{
    Console.WriteLine($"==== {i} ====");
    Thread.Sleep(1000);
}

==== 7 ====
任务开始工作... ...
==== 6 ====
==== 5 ====
==== 4 ====
==== 3 ====
任务完成，完成时候的状态为： IsCanceled=False  IsCompleted=True  IsFaulted=False
==== 2 ====
==== 1 ====



(8,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



In [3]:
public static class TaskClass
{
    public static void TaskMethod(string taskname, int duration)
    {
        Console.WriteLine($"Task [{taskname}] is start... ...");
        Console.WriteLine($"Task [{taskname}] is running on a thread id: {Thread.CurrentThread.ManagedThreadId}, is thread pool thread: {Thread.CurrentThread.IsThreadPoolThread}");
        Thread.Sleep(duration);
        Console.WriteLine($"Task [{taskname}] is stop... ...");
    }

    public static void TaskReport(Task task)
    {
        Console.WriteLine($"任务完成，完成时候的状态为： IsCanceled={task.IsCanceled}  IsCompleted={task.IsCompleted}  IsFaulted={task.IsFaulted}");
    }
    
    public static int TaskMethodInt(string taskname, int duration)
    {
        Console.WriteLine($"Task [{taskname}] is start... ...");
        Console.WriteLine($"Task [{taskname}] is running on a thread id: {Thread.CurrentThread.ManagedThreadId}, is thread pool thread: {Thread.CurrentThread.IsThreadPoolThread}");
        Thread.Sleep(duration);
        Console.WriteLine($"Task [{taskname}] is stop... ...");

        return 1024;
    }
}

In [4]:
Task t = new Task(() => TaskClass.TaskMethod("TestTask", 5000));
t.Start();
t.ContinueWith((task) => TaskClass.TaskReport(task));

//模拟另一个任务
for (int i = 7; i > 0; i--)
{
    Console.WriteLine($"==== {i} ====");
    Thread.Sleep(1000);
}

==== 7 ====
Task [TestTask] is start... ...
Task [TestTask] is running on a thread id: 15, is thread pool thread: True
==== 6 ====
==== 5 ====
==== 4 ====
==== 3 ====
Task [TestTask] is stop... ...
任务完成，完成时候的状态为： IsCanceled=False  IsCompleted=True  IsFaulted=False
==== 2 ====
==== 1 ====



(3,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.

